# Propagation des anomalies de convection profonde en zone tropicale

Auteur : FERRY Frédéric (ENM/C3M) - Novembre 2021

L'objectif de ce TP Python est d’étudier les caractéristiques de propagation des anomalies de convection profonde en zone tropicale à travers l'analyse de cartes et d'animations et l'exploitation de diagrammes temps-longitude (de Hovmöller).

On cherchera à mettre en évidence les signatures des ondes équatoriales couplées à la convection et de l'Oscillation de Madden-Julian qui sont une source de variabilité de l'atmosphère tropicale à l'échelle intra-mensuelle et intra-saisonnière.

On s'intéressera à des champs quotidiens champs d'OLR, de potentiel de vitesse à 200hPa et de vent zonal à 850hPa, et plus précisément à la propagation des anomalies de ces champs.
 
** Outgoing Longwave Radiation (OLR) : mesure par satellite de la quantité d’énergie émise dans l’infrarouge vers l’espace par la surface terrestre, les océans et l’atmosphère. (unité : W/m2) : c’est une composante essentielle du bilan radiatif terrestre.**

** Potentiel de vitesse (Velocity Potential, VP) : composante divergente/convergente de l’écoulement (unité : m2/s). Le VP est calculé en inversant le laplacien de la divergence du vent horizontal, ce qui en fait un champ lissé facile à analyser. Remarque : le VP est généralement calculé à partir de données globales (utilisation des harmoniques sphériques pour l’inversion du Laplacien), il est plus difficile à calculer pour un domaine limité.**

Concepts Python illustrés :

- Exploitation de fichiers de données quotidiennes au format netcdf
- Calcul de climatologies
- Décomposition d'un champ de vent en fonction de courant et potentiel de vitesse (module windspharm)
- Création de fichier netcdf
- Calcul d'anomalies quotidiennes
- Tracé de séries temporelles ponctuelles
- Tracé de cartes et d'animations
- Tracé de diagrammes temps-longitude

<p>
** Instructions : **
</p>

<p>
** Exécuter les cellules qui suivent de façon séquentielle **
</p>

<span style='background :yellow' > ** Répondre (directement dans le calepin) aux questions dans les parties avec titre jaune ** </span>

<p>
** Sauvegarder le notebook final au format pdf **
</p>

In [ ]:
%matplotlib inline

import os
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.util import add_cyclic_point

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dir_data='./data/'
dir_results='./result/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

In [ ]:
olr_file='olr.day.mean.nc'
u200_file='uwnd200.1980-2021.nc'
v200_file='vwnd200.1980-2021.nc'
u850_file='uwnd850.1980-2021.nc'
v850_file='vwnd850.1980-2021.nc'
vp_file='vp200.1980-2021.nc'

# Etape 1 : analyse des données d'OLR

Ouverture du fichier de données quotidiennes d'OLR : https://www.esrl.noaa.gov/psd/data/gridded/data.interp_OLR.html

In [ ]:
olr    = xr.open_dataset(dir_data+olr_file)
print(olr)

In [ ]:
year1='1980'
year2='2020'
latS=-30.
latN=30.

olr    = xr.open_dataset(dir_data+olr_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
olr_mean = olr.groupby('time.season').mean('time')
seasons=['DJF','JJA','MAM','SON']

print(olr.time.values)
lat = olr.lat.values
lon = olr.lon.values

print(olr_mean)
data = olr_mean['olr']

In [ ]:
lonW=lon[0]
lonE=lon[-1]

bounds = [(lonW, lonE, latS, latN)]

def plot_background(ax):
    ax.set_xticks(np.linspace(0, 360, 13), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(latS, latN, 5), crs=ccrs.PlateCarree())
    ax.axes.axis('tight')
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.coastlines()
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    return ax

In [ ]:
levels = np.arange(160,335,5)

axes_class = (GeoAxes, dict(map_projection=ccrs.PlateCarree(central_longitude=180.0)))
fig = plt.figure(figsize=(15, 15))
fig.suptitle('Outgoing Longwave Radiation (W/m$^2$) : '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(4, 1),
       axes_pad=0.6,
       cbar_location='bottom',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='10%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon, lat, data[i,:,:], levels, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname=dir_figs+'OLR_global_climatology'
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>


<p>
**1)** A quelles gammes de valeurs d’OLR correspondent les régions de convection profonde en zone tropicale ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

# Etape 2 : calcul du potentiel de vitesse à 200 hPa

<span style='background :red' > ** Module windspharm nécessaire, à exécuter uniquement sur Mac/Linux ** </span>

On dispose de données quotidiennes de vent zonal et de vent méridien sur la période 1948-2020 : https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.pressure.html

Grâce au  module windspharm on va créer ici un fichier de données quotidiennes de potentiel de vitesse à 200 hPa (VP200) à partir des données de vent horizontal à 200hPa et au format netcdf
--> fichier vp200.1948-2020.nc dans le répertoire result
--> VP200    (time, lat, lon)

<span style='background :red' > Attention : étape longue à ne faire qu'une fois ! Si cela ne fonctionne pas, sauter cette étape et utiliser pour la suite le fichier vp200.1948-2020.nc de secours présent dans le répertoire data. </span>

In [ ]:
from windspharm.standard import VectorWind
from windspharm.tools import prep_data, recover_data, order_latdim
from windspharm.examples import example_data_path

u    = xr.open_dataset(dir_data+u200_file)
v    = xr.open_dataset(dir_data+v200_file)

vp_file='vp200.1980-2021.nc'
sf_file='sf200.1980-2021.nc'

uwnd = u.uwnd.values
vwnd =  v.vwnd.values
times  = u.time.values
lats  = u.lat.values
lons  = u.lon.values

# The standard interface requires that latitude and longitude be the leading
# dimensions of the input wind components, and that wind components must be
# either 2D or 3D arrays. The data read in is 3D and has latitude and
# longitude as the last dimensions. The bundled tools can make the process of
# re-shaping the data a lot easier to manage.
uwnd, uwnd_info = prep_data(uwnd, 'tyx')
vwnd, vwnd_info = prep_data(vwnd, 'tyx')
print(uwnd_info)

# Create a VectorWind instance to handle the computation of streamfunction and
# velocity potential.
w = VectorWind(uwnd, vwnd)

# Compute the streamfunction and velocity potential. 
sf, vp = w.sfvp()

# Use the bundled tools to re-shape the outputs to the 4D shape of the wind components 
# as they were read off files.
sf = recover_data(sf, uwnd_info)
vp = recover_data(vp, uwnd_info)

# Convert numpy arrays to xarray DataArrays and netcdf files
vp2 = xr.DataArray(vp,dims=['time','lat','lon'], coords={'time': times,'lat': lats, 'lon': lons})
vp2.name='VP200'
print(vp2)
vp2.to_netcdf(dir_results+vp_file)

sf2 = xr.DataArray(sf,dims=['time','lat','lon'], coords={'time': times,'lat': lats, 'lon': lons})
sf2.name='SF200'
print(sf2)
sf2.to_netcdf(dir_results+sf_file)

# Etape 3 : analyse des données de potentiel de vitesse à 200 hPa

In [ ]:
year1='1980'
year2='2020'
latS=-30.
latN=30.

if not os.path.exists(dir_results+vp_file):
    vp    = xr.open_dataset(dir_data+vp_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
if os.path.exists(dir_results+vp_file):
    vp    = xr.open_dataset(dir_results+vp_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
lat = vp.lat.values
lon = vp.lon.values
print(vp)

vp_mean = vp.groupby('time.season').mean('time')
seasons=['DJF','JJA','MAM','SON']
print(vp_mean)
data = vp_mean['VP200']

In [ ]:
levels = np.arange(-10,12,2)

axes_class = (GeoAxes, dict(map_projection=ccrs.PlateCarree(central_longitude=180.0)))
fig = plt.figure(figsize=(15, 15))
fig.suptitle('Velocity potential at 200 hPa ($10^6$m$^2$s$^{-1}$) : '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(4, 1),
       axes_pad=0.6,
       cbar_location='bottom',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='10%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon, lat, data[i,:,:] * 1e-06, levels, transform=ccrs.PlateCarree(), cmap='BrBG_r', extend='both')
    axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname=dir_figs+'vp200_tropics_climatology'
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>

<p>
**1)**  Quel est le signe du potentiel de vitesse à 200hPa dans les régions de convection profonde tropicale ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**2)**  Comparer la structure spatiale du champ de potentiel de vitesse avec celle de l'OLR.
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

# Etape 4 : calcul des anomalies quotidiennes (OLR, U850, VP200).

In [ ]:
year1='1980'
year2='2020'
latS=-15.
latN=15.

olr    = xr.open_dataset(dir_data+olr_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
if not os.path.exists(dir_results+vp_file):
    vp    = xr.open_dataset(dir_data+vp_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
if os.path.exists(dir_results+vp_file):
    vp    = xr.open_dataset(dir_results+vp_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
u    = xr.open_dataset(dir_data+u850_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
v    = xr.open_dataset(dir_data+v850_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))

lat = olr.lat.values
lon = olr.lon.values

print(' ----- Computing daily anomalies ----- ')
olr_anom=olr.groupby('time.dayofyear') - olr.groupby('time.dayofyear').mean('time')
vp_anom=vp.groupby('time.dayofyear') - vp.groupby('time.dayofyear').mean('time')
u_anom=u.groupby('time.dayofyear') - u.groupby('time.dayofyear').mean('time')
v_anom=v.groupby('time.dayofyear') - v.groupby('time.dayofyear').mean('time')

print(olr_anom)
print(vp_anom)
print(u_anom)
print(v_anom)

print(' ----- Writing netcdf ----- ')
dataDIR = dir_results+'olr.anom.'+year1+'-'+year2+'.nc'
olr_anom.to_netcdf(dataDIR)
dataDIR = dir_results+'vp200.anom.'+year1+'-'+year2+'.nc'
vp_anom.to_netcdf(dataDIR)
dataDIR = dir_results+'uwnd850.anom.'+year1+'-'+year2+'.nc'
u_anom.to_netcdf(dataDIR)
dataDIR = dir_results+'vwnd850.anom.'+year1+'-'+year2+'.nc'
v_anom.to_netcdf(dataDIR)
print(' ----- Done ----- ')

# Etape 5 : analyse des anomalies quotidiennes (OLR, U850, VP200).

In [ ]:
date1='2005-01-01'
date2='2005-12-31'

olr_days=olr.sel(time=slice(date1,date2)).sel(lat=0, method='nearest').sel(lon=120, method='nearest')
olr_days_anom=olr_anom.sel(time=slice(date1,date2)).sel(lat=0, method='nearest').sel(lon=120, method='nearest')

vp_days=vp.sel(time=slice(date1,date2)).sel(lat=0, method='nearest').sel(lon=120, method='nearest') * 1e-06
vp_days_anom=vp_anom.sel(time=slice(date1,date2)).sel(lat=0, method='nearest').sel(lon=120, method='nearest') * 1e-06

print(olr_days)
print(olr_days_anom)

data1_olr = olr_days['olr']
data2_olr = olr_days_anom['olr']
data3_olr=data1_olr-data2_olr

data1_vp = vp_days['VP200']
data2_vp = vp_days_anom['VP200']
data3_vp=data1_vp-data2_vp

In [ ]:
plt_title = 'OLR and VP200 at (0,120E) : '+date1+'-'+date2

fig=plt.figure(figsize=(20, 15))
fig.suptitle(plt_title, fontsize=16)

ax = plt.subplot(211)
plt.title('OLR', fontsize=12)
x = [i for i in range(0, 365,1)]
plt.xlabel('days of the year')
plt.ylabel('W/m$^2$')
clim = ax.plot(data3_olr, color='black', linewidth=1, linestyle='-', label='Daily climatology')
total = ax.plot(data1_olr, color='green', linewidth=1, linestyle='-', label='Daily field')
ax.fill_between(x, data3_olr, data1_olr, where=data1_olr >= data3_olr, facecolor='red', interpolate=True)
ax.fill_between(x, data3_olr, data1_olr, where=data1_olr <= data3_olr, facecolor='blue', interpolate=True)
plt.legend()

ax = plt.subplot(212)
plt.title('VP200', fontsize=12)
x = [i for i in range(0, 365,1)]
plt.xlabel('days of the year')
plt.ylabel('$10^6$m$^2$s$^{-1}$')
clim = ax.plot(data3_vp, color='black', linewidth=1, linestyle='-', label='Daily climatology')
total = ax.plot(data1_vp, color='green', linewidth=1, linestyle='-', label='Daily field')
ax.fill_between(x, data3_vp, data1_vp, where=data1_vp >= data3_vp, facecolor='red', interpolate=True)
ax.fill_between(x, data3_vp, data1_vp, where=data1_vp <= data3_vp, facecolor='blue', interpolate=True)
plt.legend()

plt.show()

figname=dir_figs+'OLR_VP200_Maritime_Continent'+date1+'-'+date2
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>

<p>
**1)**  Commenter les cycles annuel moyens de l'OLR et du VP200 sur le point choisi (courbe noire).
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**2)**  Quels sont les ordres de grandeurs des anomalies d'OLR et de VP200 sur le point choisi (zones en rouge/bleu) ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

In [ ]:
date1='2005-04-01'

olr_day=olr.sel(time=date1)
olr_day_anom=olr_anom.sel(time=date1)

vp_day=vp.sel(time=date1) * 1e-06
vp_day_anom=vp_anom.sel(time=date1) * 1e-06

u_day=u.sel(time=date1)
u_day_anom=u_anom.sel(time=date1)
v_day=v.sel(time=date1)
v_day_anom=v_anom.sel(time=date1)

print(olr_day)
olr_data1 = olr_day['olr']
olr_data2 = olr_day_anom['olr']
vp_data1 = vp_day['VP200']
vp_data2 = vp_day_anom['VP200']
u_data1 = u_day['uwnd']
u_data2 = u_day_anom['uwnd']
v_data1 = v_day['vwnd']
v_data2 = v_day_anom['vwnd']

In [ ]:
u_data1=np.array(u_data1)
v_data1=np.array(v_data1)
u_data2=np.array(u_data2)
v_data2=np.array(v_data2)

In [ ]:
olr_day_anom_avg=olr_day_anom.sel(lat=slice(latN,latS)).mean('lat')
vp_day_anom_avg=vp_day_anom.sel(lat=slice(latN,latS)).mean('lat')
print(olr_day_anom_avg)
olr_profile = olr_day_anom_avg['olr']
vp_profile = vp_day_anom_avg['VP200']

In [ ]:
olr_levels1 = np.arange(80,320,10)
olr_levels2 = np.arange(-160,180,20)
vp_levels1 = np.arange(-15,17.5,2.5)
vp_levels2 = np.arange(-15,17.5,2.5)
olr_cmap1='jet'
olr_cmap2='RdBu_r'
vp_cmap1='BrBG_r'
vp_cmap2='BrBG_r'

bounds = [(lonW, lonE, latS, latN)]

fig = plt.figure(figsize=(15., 10.))
fig.suptitle('VP200 : '+date1, fontsize=16)
ax = fig.add_subplot(3, 1, 1, projection=ccrs.PlateCarree(central_longitude=180.0))
ax.set_title('Daily field - VP200', loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, vp_data1, vp_levels1, transform=ccrs.PlateCarree(), cmap=vp_cmap1, extend='both')
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')

ax = fig.add_subplot(3, 1, 2, projection=ccrs.PlateCarree(central_longitude=180.0))
ax.set_title('Anomaly field - VP200', loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, vp_data2, vp_levels2, transform=ccrs.PlateCarree(), cmap=vp_cmap2, extend='both')

cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')

ax = fig.add_subplot(3,1,3)
plt.title('Anomaly field - VP200 : 15S-15N average', fontsize=12)
ax.set_xticks(np.linspace(0, 360, 13))
ax.set_xticklabels(np.linspace(0, 360, 13))
plt.xlim(lon[0], lon[-1])
plt.xlabel('Longitude')
plt.ylabel('Anomaly ($10^6$m$^2$s$^{-1}$)')
prof = ax.plot(lon, vp_profile, color='black', linewidth=1, linestyle='-')
plt.axhline(0, color='k')
plt.axvline(0, color='k', linestyle='--')
plt.axvline(90, color='k', linestyle='--')
plt.axvline(180, color='k', linestyle='--')
plt.axvline(270, color='k', linestyle='--')

plt.show()

figname=dir_figs+'VP200_'+date1
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>

<p>
**1)** Pour le jour choisi, localiser les principales régions favorables/défavorables à grande échelle au développpemnt de la convection profonde.
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

In [ ]:
fig = plt.figure(figsize=(15., 10.))
fig.suptitle('OLR and winds at 850 hPa : '+date1, fontsize=16)
ax = fig.add_subplot(3, 1, 1, projection=ccrs.PlateCarree(central_longitude=180.0))
ax.set_title('Daily field - OLR and winds', loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, olr_data1, olr_levels1, transform=ccrs.PlateCarree(), cmap=olr_cmap1, extend='both')
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
wind = ax.quiver(lon, lat, u_data1, v_data1, scale=600, angles='xy', regrid_shape=10, transform=ccrs.PlateCarree())

ax = fig.add_subplot(3, 1, 2, projection=ccrs.PlateCarree(central_longitude=180.0))
ax.set_title('Anomaly field - OLR and winds', loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, olr_data2, olr_levels2,  transform=ccrs.PlateCarree(), cmap=olr_cmap2, extend='both')

cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
wind = ax.quiver(lon, lat, u_data2, v_data2, scale=600, angles='xy', regrid_shape=10, transform=ccrs.PlateCarree())

ax = fig.add_subplot(3,1,3)
plt.title('Anomaly field - OLR : 15S-15N average', fontsize=12)
ax.set_xticks(np.linspace(0, 360, 13))
ax.set_xticklabels(np.linspace(0, 360, 13))
plt.xlim(lon[0], lon[-1])
plt.xlabel('Longitude')
plt.ylabel('Anomaly (W/m$^2$)')
prof = ax.plot(lon, olr_profile, color='black', linewidth=1, linestyle='-')
plt.axhline(0, color='k')
plt.axvline(0, color='k', linestyle='--')
plt.axvline(90, color='k', linestyle='--')
plt.axvline(180, color='k', linestyle='--')
plt.axvline(270, color='k', linestyle='--')
plt.show()

figname=dir_figs+'OLR_wind_'+date1
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>


<p>
**1)**  Pour le jour choisi, localiser les principales anomalies (renforcement, affaiblissement) de convection profonde.

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**2)** Quelle est le signe dominant de l'anomalie de vent zonal sur l'Océan Indien?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

In [ ]:
date1='2005-04-01'
date2='2005-04-30'

olr_days=olr.sel(time=slice(date1,date2))
olr_days_anom=olr_anom.sel(time=slice(date1,date2))
vp_days=vp.sel(time=slice(date1,date2)) * 1e-06
vp_days_anom=vp_anom.sel(time=slice(date1,date2)) * 1e-06
u_days=u.sel(time=slice(date1,date2))
u_days_anom=u_anom.sel(time=slice(date1,date2))
v_days=v.sel(time=slice(date1,date2))
v_days_anom=v_anom.sel(time=slice(date1,date2))

print(olr_days_anom)

time  = olr_days.time.values
lat = olr_days.lat.values

olr_data1 = olr_days['olr']
olr_data2 = olr_days_anom['olr']
vp_data1 = vp_days['VP200']
vp_data2 = vp_days_anom['VP200']
u_data1 = u_days['uwnd']
u_data2 = u_days_anom['uwnd']
v_data1 = v_days['vwnd']
v_data2 = v_days_anom['vwnd']

In [ ]:
time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]

for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]

print(date_str)

In [ ]:
u_data1=np.array(u_data1)
v_data1=np.array(v_data1)
u_data2=np.array(u_data2)
v_data2=np.array(v_data2)

In [ ]:
olr_days_anom_avg=olr_days_anom.sel(lat=slice(latN,latS)).mean('lat')
vp_days_anom_avg=vp_days_anom.sel(lat=slice(latN,latS)).mean('lat')
print(olr_days_anom_avg)
olr_profiles = olr_days_anom_avg['olr']
vp_profiles = vp_days_anom_avg['VP200']
print(vp_profiles)

In [ ]:
plt_title = 'VP200 : '+date1+'-'+date2

for i in range(len(time)):
    print(date_str[i])
    fig = plt.figure(figsize=(15., 10.))
    fig.suptitle('VP200 : '+date_str[i], fontsize=16)
    
    ax = fig.add_subplot(3, 1, 1, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('Daily field', loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, vp_data1[i,:,:], vp_levels1, transform=ccrs.PlateCarree(),
                     cmap=vp_cmap1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    ax = fig.add_subplot(3, 1, 2, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('Anomaly field', loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, vp_data2[i,:,:], vp_levels2, transform=ccrs.PlateCarree(),
                     cmap=vp_cmap2, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    ax = fig.add_subplot(3,1,3)
    plt.title('Anomaly field : 15S-15N average', fontsize=12)
    ax.set_xticks(np.linspace(0, 360, 13))
    ax.set_xticklabels(np.linspace(0, 360, 13))
    plt.xlim(lon[0], lon[-1])
    plt.ylim(-18, 18)    
    plt.xlabel('Longitude')
    plt.ylabel('Anomaly ($10^6$m$^2$s$^{-1}$)')
    prof = ax.plot(lon, vp_profiles[i,:], color='black', linewidth=1, linestyle='-')
    plt.axhline(0, color='k')
    plt.axvline(0, color='k', linestyle='--')
    plt.axvline(90, color='k', linestyle='--')
    plt.axvline(180, color='k', linestyle='--')
    plt.axvline(270, color='k', linestyle='--')
    plt.close()
    figname=dir_anim+'VP200_'+date_str[i]
    fig.savefig(figname+'.png')

In [ ]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format(dir_anim)))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir(dir_anim)
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
gif_filepath = dir_anim+'VP200_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# <span style='background :yellow' > Questions </span>

<p>
**1)** Analyser la chronologie de la propagation des anomalies de VP200 au cours du mois choisi. Quel est le mode de propagation principal qui semble s'exprimer ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

In [ ]:
for i in range(len(time)):
    print(date_str[i])
    fig = plt.figure(figsize=(15., 10.))
    fig.suptitle('OLR and winds at 850 hPa : '+date_str[i], fontsize=16)
    
    ax = fig.add_subplot(3, 1, 1, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('Daily field - OLR and winds', loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, olr_data1[i,:,:], olr_levels1, transform=ccrs.PlateCarree(),
                     cmap=olr_cmap1, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    wind = ax.quiver(lon, lat, u_data1[i,:,:], v_data1[i,:,:], scale=600, angles='xy', regrid_shape=10,
                     transform=ccrs.PlateCarree())
    
    ax = fig.add_subplot(3, 1, 2, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('Anomaly field - OLR and winds', loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, olr_data2[i,:,:], olr_levels2, transform=ccrs.PlateCarree(),
                     cmap=olr_cmap2, extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    wind = ax.quiver(lon, lat, u_data2[i,:,:], v_data2[i,:,:], scale=600, angles='xy', regrid_shape=10,
                     transform=ccrs.PlateCarree())
    
    ax = fig.add_subplot(3,1,3)
    plt.title('Anomaly field - OLR : 15S-15N average', fontsize=12)
    ax.set_xticks(np.linspace(0, 360, 13))
    ax.set_xticklabels(np.linspace(0, 360, 13))
    plt.ylim(-80, 80)
    plt.xlim(lon[0], lon[-1])
    plt.xlabel('Longitude')
    plt.ylabel('Anomaly (W/m$^2$)')
    prof = ax.plot(lon, olr_profiles[i,:], color='black', linewidth=1, linestyle='-')
    plt.axhline(0, color='k')
    plt.axvline(0, color='k', linestyle='--')
    plt.axvline(90, color='k', linestyle='--')
    plt.axvline(180, color='k', linestyle='--')
    plt.axvline(270, color='k', linestyle='--')
    figname=dir_anim+'OLR_wind_'+date_str[i]
    fig.savefig(figname+'.png')
    plt.close()

In [ ]:
gif_filepath = dir_anim+'OLR_wind_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# <span style='background :yellow' > Questions </span>


<p>
**1)** Analyser le déplacement de la principale zone de convection profonde. Quelles différences de propagation peut-on observer par rapport aux données de VP200 ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

# Etape 6 : réalisation de diagrammes temps-longitude (OLR, U850, VP200).

In [ ]:
date1=input('Entrer la date de début au format YYYY-MM-DD : ')
date2=input('Entrer la date de fin au format YYYY-MM-DD : ')

olr_days=olr.sel(time=slice(date1,date2))
olr_days_anom=olr_anom.sel(time=slice(date1,date2))
vp_days=vp.sel(time=slice(date1,date2)) * 1e-06
vp_days_anom=vp_anom.sel(time=slice(date1,date2)) * 1e-06
u_days=u.sel(time=slice(date1,date2))
u_days_anom=u_anom.sel(time=slice(date1,date2))
lat = olr_days.lat.values
print(olr_days_anom)

time_hov  = olr_days.time.values
time_str=[x for x in range(len(time_hov))]
date_str=[x for x in range(len(time_hov))]

for i in range(len(time_hov)):
	time_str[i] = str(time_hov[i])
	date_str[i] = time_str[i][0:10]
print(date_str)

In [ ]:
olr_days_anom_avg=olr_days_anom.sel(lat=slice(latN,latS)).mean('lat')
vp_days_anom_avg=vp_days_anom.sel(lat=slice(latN,latS)).mean('lat')
u_days_anom_avg=u_days_anom.sel(lat=slice(latN,latS)).mean('lat')

print(olr_days_anom_avg)

lon_hov=olr_days_anom_avg.lon.values
lon1=lon_hov[0]
lon2=lon_hov[-1]

data_hov_olr=olr_days_anom_avg['olr']
data_hov_vp=vp_days_anom_avg['VP200']
data_hov_u=u_days_anom_avg['uwnd']

print(data_hov_olr)

In [ ]:
olr_levels = np.arange(-100,110,10)
vp_levels = np.arange(-18,20,2)
u_levels = np.arange(-10,11,1)

##### VP200
fig = plt.figure(figsize=(12., 15.))
ax=plt.subplot(1, 1, 1)
plt.gca().invert_yaxis()
ax.set_yticks(time_hov[::4])
ax.set_yticklabels(date_str[::4])
ax.set_xticklabels('')
cf = ax.contourf(lon_hov, time_hov, data_hov_vp, vp_levels, cmap='BrBG_r', extend='both')

cb = fig.colorbar(cf, orientation='vertical', extend='max', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('$10^6$m$^2$s$^{-1}$', size='large')
ax.set_title('VP200 anomaly : '+date1+'-'+date2, loc='center')

ax_inset = fig.add_axes([0.125, 0.03, 0.62, 0.05], projection=ccrs.PlateCarree(central_longitude=180.0))
bounds = [(lon1, lon2, latS, latN)]
ax_inset.axes.axis('tight')
ax_inset.set_yticks(np.linspace(latS, latN, 2), crs=ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax_inset.yaxis.set_major_formatter(lat_formatter)
ax_inset.set_xticks(np.linspace(-180, 180, 13))
ax_inset.set_xticklabels(np.linspace(-180, 180, 13))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax_inset.xaxis.set_major_formatter(lon_formatter)
ax_inset.coastlines()
ax_inset.stock_img()
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())

plt.show()

figname='./figs/VP_Hov_'+date1+'-'+date2
fig.savefig(figname+'.png')

##### OLR
fig = plt.figure(figsize=(12., 15.))
ax=plt.subplot(1, 1, 1)
plt.gca().invert_yaxis()
ax.set_yticks(time_hov[::4])
ax.set_yticklabels(date_str[::4])
ax.set_xticklabels('')
cf = ax.contourf(lon_hov, time_hov, data_hov_olr, olr_levels, cmap='RdBu_r', extend='both')

cb = fig.colorbar(cf, orientation='vertical', extend='max', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('W/m$^2$', size='large')
ax.set_title('OLR anomaly : '+date1+'-'+date2, loc='center')

ax_inset = fig.add_axes([0.125, 0.03, 0.62, 0.05], projection=ccrs.PlateCarree(central_longitude=180.0))
bounds = [(lon1, lon2, latS, latN)]
ax_inset.axes.axis('tight')
ax_inset.set_yticks(np.linspace(latS, latN, 2), crs=ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax_inset.yaxis.set_major_formatter(lat_formatter)
ax_inset.set_xticks(np.linspace(-180, 180, 13))
ax_inset.set_xticklabels(np.linspace(-180, 180, 13))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax_inset.xaxis.set_major_formatter(lon_formatter)
ax_inset.coastlines()
ax_inset.stock_img()
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())

plt.show()

figname=dir_figs+'OLR_Hov_'+date1+'-'+date2
fig.savefig(figname+'.png')

##### U850
fig = plt.figure(figsize=(12., 15.))
ax=plt.subplot(1, 1, 1)
plt.gca().invert_yaxis()
ax.set_yticks(time_hov[::4])
ax.set_yticklabels(date_str[::4])
ax.set_xticklabels('')
cf = ax.contourf(lon_hov, time_hov, data_hov_u, u_levels, cmap='RdBu_r', extend='both')

cb = fig.colorbar(cf, orientation='vertical', extend='max', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('m/s', size='large')
ax.set_title('U850 anomaly : '+date1+'-'+date2, loc='center')

ax_inset = fig.add_axes([0.125, 0.03, 0.62, 0.05], projection=ccrs.PlateCarree(central_longitude=180.0))
bounds = [(lon1, lon2, latS, latN)]
ax_inset.axes.axis('tight')
ax_inset.set_yticks(np.linspace(latS, latN, 2), crs=ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax_inset.yaxis.set_major_formatter(lat_formatter)
ax_inset.set_xticks(np.linspace(-180, 180, 13))
ax_inset.set_xticklabels(np.linspace(-180, 180, 13))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax_inset.xaxis.set_major_formatter(lon_formatter)
ax_inset.coastlines()
ax_inset.stock_img()
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())

plt.show()

figname=dir_figs+'U850_Hov_'+date1+'-'+date2
fig.savefig(figname+'.png')

# <span style='background :yellow' > Questions </span>

<p>
**1)** Caractériser la propagation principale vers l'est des anomalies d’OLR et de VP200 en termes de vitesses de phase. Cette vitesse de phase est-elle la même pour toutes les longitudes ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**2)** Quel est le signe de l’anomalie de vent zonal associée à la propagation vers l’est des zones de renforcement de la convection profonde ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**3)** Ces diagrammes confirment-ils les différences de propagations entre anomalies d'OLR et anomalies de VP200 constatées sur les cartes ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>

<p>
**4)** Réaliser les diagrammes de Hovmoller pour la période janvier-mars 2000. Comment qualifier la propagation des anomalies de convection profonde sur cette période ?
</p>

<p style="background:rgb(230,255,230)">
votre réponse ici
</p>